# 主题模型

### 1.课堂示例 2.个人代码演示

LDA（Latent Dirichlet Allocations）主题模型又叫潜在狄利克雷分布模型，是一种非监督学习算法，包括文档-主题和主题-词语概率分布。该模型通过分析非结构化文本数据中的词语信息对一系列文档中的主题进行抽象和聚类，从而实现文本分类，也能够发掘语料库中英藏的主题模式，根据主题对文档进行标注并利用标注对文档进行组织、整理、归纳和检索。

LDA模型求解算法主要有两种，即EM算法和Gibbs采样。



1.EM算法



由于LDA模型中含有隐变量（某篇文档中某个单词的所属主题Zi,j)，LDA模型的似然函数不存在显示的表达式。因此，一般采用的最大化对数似然函数的概率模型求解算法LDA并不适用。EM算法，全称Expectation-Maximum算法，是一种计算含有隐变量的概率模型的最常用算法之一。EM算法并不直接计算对数似然函数的最值，而是计算对数似然函数对隐变量的期望，并通过最大化该期望来计算模型的参数。EM算法是一种迭代算法，在给定待求解模型参数的初始值后，不断迭代E-Step和M-Step两个步骤来寻找最佳的模型参数。



（1）E-Step：已知当前迭代的模型参数，计算出对数似然函数对隐变量的期望（或者只求解对数似然函数中包含期望的部分即可）。



（2）M-Step：求解E-Step中所得期望的最大值，得到新的模型参数。



2.Gibbs采样



Gibbs采样是一种求解高维概率模型的常用迭代算法。Gibbs采样的思路是，每次迭代中只选取概率向量的一个维度进行求解，即固定其他维度的变量值采样当前维度的值。不断迭代，直到收敛输出待估计的参数。LDA模型中，Gibbs采样的计算方法如下：



初始时随机给文本中的每个单词分配主题Zi,j，然后统计每个主题Zi,j下出现单词w的数量以及每个文档i下出现主题Zi,j中的词的数量，每一轮计算p(Zi,j|Z-i,j,i,w),即排除当前词的主题分配，根据其他所有词的主题分配估计当前词分配各个主题的概率。



当得到当前词属于所有主题的概率分布后，根据这个概率分布为该词采样一个新的主题Zi,j。然后用同样的方法不断更新下一个词的主题，直到发现每个文档下主题比例和每个主题下词的分布收敛，算法停止，输出待估计的模型参数，最终每个单词的主题Zi,j也同时得出。

LDA在 2003年由David Blei、Andrew Ng 和 Michael Jordan共同提出，作为一种解决PLSA潜在缺陷的方法。他们的论文 "Latent Dirichlet Allocation" 发表在Journal of Machine Learning Research 上，成为了机器学习和自然语言处理领域的经典论文。

以下是LDA的发展历程：

1.贝叶斯框架：与PLSA不同，LDA在模型中引入了狄利克雷分布作为主题和文档的先验，这使得模型更为稳健和具有泛化能力。

2.Gibbs抽样和变分推断：为了从数据中估计LDA模型的参数，研究者引入了Gibbs抽样和变分推断等方法。

3.LDA与PLSA的对比：LDA与PLSA都旨在从文档中提取潜在的主题，但LDA通过引入狄利克雷先验，在泛化到新的文档和大型数据集时具有更好的性能。

4.LDA的进一步扩展：随着LDA的流行，许多研究者对其进行了扩展，引入了时间、作者等其他因素，从而使模型更为复杂和灵活。

总的来说，LDA为文本挖掘和主题建模领域带来了革命性的改进。它不仅解决了PLSA的一些局限性，而且由于其优雅的数学形式和强大的建模能力，迅速成为了这一领域的金标准。通过深入学习LDA，您将更好地理解文本数据的内在结构，以及如何使用先验知识和概率方法来揭示这些结构。

# 1.课堂示例

In [1]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [2]:
df = pd.read_excel('/Users/jiaojiao/Downloads/text_analysis_weibo_sample.xlsx', index_col = 0)

In [3]:
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


### 语料预处理

#### 剔除符号与数字

In [6]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)[:100]

'国债地产行业重磅利好提振风险偏好期债低开低走国债期货全线收跌年期主力合约跌年期主力合约跌年期主力合约跌三大主力合约均创逾一年收盘新低行情解读公开市场方面央行开展了亿元天期逆回购操作中标利率亿元逆回购到'

#### 分词

In [7]:
# 加载中文停用词词典，可个性化设置
# stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题/微博内容'][0]
clean_text(test)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/4l/_lcd9phj5pz4r5t7ryzwbtzh0000gn/T/jieba.cache
Loading model cost 0.279 seconds.
Prefix dict has been built successfully.


'    国债 ： 地产 行业 重磅 利好 提振 风险 偏好 ， 期债 低开 低 走           国债 期货 全线 收跌 ， 10 年期 主力 合约 跌 0.37% ， 5 年期 主力 合约 跌 0.24% ， 2 年期 主力 合约 跌 0.11% ， 三大 主力 合约 均 创 逾 一年 收盘 新低 。         行情 解读 ：         1 ， 公开市场 方面 ， 央行 开展 了 800 亿元 7 天期 逆 回购 操作 ， 中标 利率 2.0% ， 20 亿元 逆 回购 到期 ， 因此 当日 净 投放 780 亿元 。         2 ， 资金面 方面 ， 央行 公开市场 逆 回购 继续 加力 ， 不过 银行 间 市场 周二 资金 供给 趋于 收敛 ， 七天 回购 加权 利率 进一步 上行 。 长期 资金 方面 ， 全国 和 主要 股份制 银行 一年期 同业 存单 报价 仍 维持 在 2.45% 水平 ， 不过 目前 尚无 成交量 配合 。         3 ， 房地产 行业 再度 迎来 重磅 利好 。 证监会 宣布 在 涉房 企业 股权 融资 方面 调整 优化 五项 措施 ， 并 自即日起 施行 。 随着 证监会 支持 房企 股权 融资 这 “ 第三支 箭 ” 正式 发射 ， 信贷 、 债券 、 股权 三个 融资 渠道 “ 三箭 齐发 ” ， 合力 推动 房地产 融资 。 — — “ 第三支 箭 ” 快速 落地 ， 或 将 为 房地产 行业 注入 较 大规模 的 增量 资金 和 资产 ， 促进 行业 平稳 健康 发展 。 在 短短 20 天 的 时间 内 政策 端 “ 三箭 齐发 ” 强势 托底 ， 提振 各方 的 信心 。 看好 全 行业 的 信用 复苏 以及 在 因城 施策 空间 逐步 打开 下 的 销售 回暖 。         4 ， 国务院 联防 联控 机制 11 月 29 日 15 : 00 召开 新闻 发布会 。 针对 “ 考虑 到 最近 个别 地方 的 不满情绪 ， 政府 是否 会 重新考虑 疫情 应对 政策 ” 的 问题 ， 国家 卫生 健康 委 新闻 发言人 、 宣传司 副司长 米锋 称 ， 疫情 发生 以来 ， 中国政府 一直 根据 病毒 变异 的 特点 和 临床 治疗 的 实践 认识 ， 边 防控 、 边 研究 

In [9]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他,"[国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 低, 走,..."
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西,"[喜迎, 二十大, 忠诚, 保平安]"
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建,"[注意, 事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告,..."
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京,"[近日, 千年, 大计, 雄安, 新区, 迎来, 五周岁, 生日, 从, 一张白纸, 到, ..."
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东,"[樊振东, 牛, 逼]"
...,...,...,...,...,...,...,...,...,...,...
95,14034,老公好漂亮[舔屏],0,0,0,aa7af97ae98cbe8983e559b948ecfabf,825,191,北京,"[老公, 好, 漂亮, 舔, 屏]"
96,35249,恭喜@张小娜呀娜 1名用户获得【小飞T】。C官方唯一抽奖工具@C抽奖平台 对本次抽奖进行监督...,0,0,2,f538513e5801c275cbcf285517a8ee62,51709,20,北京,"[恭喜, 张小娜, 呀, 娜, 名, 用户, 获得, 小飞, 官方, 唯一, 抽奖, 工具,..."
97,39598,接不动了，居民部门负债率提升空间太有限了。现在动辄几万一平米，总价高，月供高，预期转差，还怎...,0,0,0,2e76c39b6665a0b6f5bdc4c35f36f573,0,59,四川,"[接不动, 了, 居民, 部门, 负债率, 提升, 空间, 太, 有限, 了, 现在, 动辄..."
98,48977,目前电子烟政策已从制定阶段逐渐走向实施阶段，而主要的电子烟公司股价下跌幅度高达70%-...,0,0,0,NaN,0,0,北京,"[目前, 电子, 烟, 政策, 已, 从, 制定, 阶段, 逐渐, 走向, 实施, 阶段, ..."


### LDA

In [10]:
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

In [11]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [12]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.018*"的" + 0.014*"月" + 0.013*"万吨" + 0.009*"市场" + 0.008*"日"')
(1, '0.033*"的" + 0.010*"考生" + 0.008*"在" + 0.006*"月" + 0.006*"加息"')
(2, '0.031*"的" + 0.015*"电子" + 0.014*"烟" + 0.011*"年" + 0.008*"在"')
(3, '0.015*"的" + 0.006*"重庆" + 0.005*"了" + 0.004*"山娃" + 0.004*"中国"')
(4, '0.037*"的" + 0.010*"在" + 0.008*"了" + 0.007*"郑州" + 0.006*"月"')


In [13]:
df['微博内容分词'].iloc[0]

['国债',
 '地产',
 '行业',
 '重磅',
 '利好',
 '提振',
 '风险',
 '偏好',
 '期债',
 '低开',
 '低',
 '走',
 '国债',
 '期货',
 '全线',
 '收跌',
 '年期',
 '主力',
 '合约',
 '跌',
 '年期',
 '主力',
 '合约',
 '跌',
 '年期',
 '主力',
 '合约',
 '跌三大',
 '主力',
 '合约',
 '均',
 '创',
 '逾',
 '一年',
 '收盘',
 '新低',
 '行情',
 '解读',
 '公开市场',
 '方面',
 '央行',
 '开展',
 '了',
 '亿元',
 '天期',
 '逆',
 '回购',
 '操作',
 '中标',
 '利率',
 '亿元',
 '逆',
 '回购',
 '到期',
 '因此',
 '当日',
 '净',
 '投放',
 '亿元',
 '资金面',
 '方面',
 '央行',
 '公开市场',
 '逆',
 '回购',
 '继续',
 '加力',
 '不过',
 '银行',
 '间',
 '市场',
 '周二',
 '资金',
 '供给',
 '趋于',
 '收敛',
 '七天',
 '回购',
 '加权',
 '利率',
 '进一步',
 '上行',
 '长期',
 '资金',
 '方面',
 '全国',
 '和',
 '主要',
 '股份制',
 '银行',
 '一年期',
 '同业',
 '存单',
 '报价',
 '仍',
 '维持',
 '在',
 '水平',
 '不过',
 '目前',
 '尚无',
 '成交量',
 '配合',
 '房地产',
 '行业',
 '再度',
 '迎来',
 '重磅',
 '利好',
 '证监会',
 '宣布',
 '在',
 '涉房',
 '企业',
 '股权',
 '融资',
 '方面',
 '调整',
 '优化',
 '五项',
 '措施',
 '并',
 '自即日起',
 '施行',
 '随着',
 '证监会',
 '支持',
 '房企',
 '股权',
 '融资',
 '这',
 '第三支',
 '箭',
 '正式',
 '发射',
 '信贷',
 '债券',
 '股权',
 '三个',
 '融资',
 '渠道',
 '三箭',
 '齐发',
 '合

In [14]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9988391399383545	 
Topic: 0.037*"的" + 0.010*"在" + 0.008*"了" + 0.007*"郑州" + 0.006*"月" + 0.006*"是" + 0.006*"和" + 0.006*"中国" + 0.005*"经济" + 0.005*"也"


In [15]:
documents = df['微博内容分词'].values

In [16]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(4, 0.9988798)]

Document 2:
[(0, 0.040013816), (1, 0.040016487), (2, 0.040015418), (3, 0.839939), (4, 0.04001523)]

Document 3:
[(1, 0.47830454), (2, 0.51974154)]

Document 4:
[(2, 0.99933594)]

Document 5:
[(0, 0.050049957), (1, 0.050850477), (2, 0.05058909), (3, 0.050073043), (4, 0.7984374)]

Document 6:
[(0, 0.022444969), (1, 0.91065013), (2, 0.022251299), (3, 0.022303313), (4, 0.022350304)]

Document 7:
[(0, 0.012598868), (1, 0.012618671), (2, 0.01260124), (3, 0.0125621585), (4, 0.94961905)]

Document 8:
[(0, 0.5973372), (1, 0.101757325), (2, 0.10020152), (3, 0.100007586), (4, 0.100696385)]

Document 9:
[(1, 0.97200125)]

Document 10:
[(4, 0.9993375)]



### 可视化 

In [17]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

In [18]:
# 导出可视化结果到html
# pyLDAvis.save_html(lda_vis, 'lda_visualization.html')

# 2.个人代码演示

In [30]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [20]:
df = pd.read_excel('/Users/jiaojiao/Downloads/政府工作报告.xlsx', index_col = 0)

In [21]:
df.head()

,report
year,
2023,新华社北京3月14日电政府工作报告 ——2023年3月5日在第十四届全国人民代表大会第一...
2022,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协委员提出意见。 ...
2021,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协委员提出意见。 ...
2020,新华社北京5月29日电 政府工作报告 ——2020年5月22日在第十三届全国人民代表...
2019,新华社北京3月16日电 政府工作报告 ——2019年3月5日在第十三届全国人民代表...


### 预料预处理

#### 剔除符号和文字

In [38]:
import re

def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums

# 确保DataFrame不为空
if df.shape[0] > 0:
    # 使用iloc访问第一行的'report'列
    test = df.iloc[0]['report']
    result = remove_nums(test)[:100]
    print(result)
else:
    print("DataFrame为空。")

新华社北京月日电政府工作报告年月日在第十四届全国人民代表大会第一次会议上国务院总理李克强月日李克强代表国务院在十四届全国人大一次会议上作政府工作报告新华社记者饶爱民摄各位代表本届政府任期即将结束现在我


### 分词

In [40]:
import jieba
# stopwords = open('stopwords.txt', encoding='utf-8').read().splitlines()

def clean_text(text):
    words = jieba.lcut(text)
    return ' '.join(words)

# 确保DataFrame不为空
if df.shape[0] > 0:
    # 使用iloc访问第一行的'report'列
    test = df.iloc[0]['report']
    result = clean_text(test)
    print(result)
else:
    print("DataFrame为空。")

　 　 新华社 北京 3 月 14 日电 政府 工作 报告   — — 2023 年 3 月 5 日 在 第十四届 全国人民代表大会 第一次 会议 上   国务院 总理 　 李克强     　 　 3 月 5 日 ， 李克强 代表 国务院 在 十四届 全国人大 一次 会议 上作 《 政府 工作 报告 》 。 新华社 记者   饶 爱民   摄   各位 代表 ： 　 　 本届 政府 任期 即将 结束 。 现在 ， 我 代表 国务院 ， 向 大会 报告 工作 ， 请予 审议 ， 并 请 全国政协 委员 提出 意见 。 　 　 一 、 过去 一年 和 五年 工作 回顾   　 　 2022 年 是 党和国家 历史 上 极为重要 的 一年 。 党 的 二十大 胜利 召开 ， 描绘 了 全面 建设 社会主义 现代化 国家 的 宏伟蓝图 。 面对 风高浪 急 的 国际 环境 和 艰巨 繁重 的 国内 改革 发展 稳定 任务 ， 以 习近平 同志 为 核心 的 党中央 团结 带领 全国 各族人民 迎难而上 ， 全面落实 疫情 要 防住 、 经济 要 稳住 、 发展 要 安全 的 要求 ， 加大 宏观调控 力度 ， 实现 了 经济 平稳 运行 、 发展 质量 稳步 提升 、 社会 大局 保持稳定 ， 我国 发展 取得 来 之 极为 不易 的 新 成就 。 　 　 过去 一年 ， 我国 经济 发展 遇到 疫情 等 国内外 多重 超 预期 因素 冲击 。 在 党中央 坚强 领导 下 ， 我们 高效 统筹 疫情 防控 和 经济社会 发展 ， 根据 病毒 变化 和 防疫 形势 ， 优化 调整 疫情 防控 措施 。 面对 经济 新 的 下行 压力 ， 果断 应对 、 及时 调控 ， 动用 近年 储备 的 政策 工具 ， 靠前 实施 既定政策 举措 ， 坚定不移 推进 供给 侧 结构性 改革 ， 出台 实施 稳 经济 一揽子 政策 和 接续 措施 ， 部署 稳住 经济 大盘 工作 ， 加强 对 地方 落实政策 的 督导 服务 ， 支持 各地 挖掘 政策 潜力 ， 支持 经济 大省勇 挑大梁 ， 突出 稳 增长 稳 就业 稳 物价 ， 推动 经济 企稳 回升 。 全年 国内 生产总值 增长 3% ， 城镇 新增 就业 1206 万人 ， 年末 城镇 调查 失业率 降到 5.5% ， 居民消费 

In [41]:
df['report'] = df['report'].astype(str)
df['report words'] = df['report'].apply(remove_nums)
df['report words'] = df['report words'].apply(clean_text)
df['report words'] = df['report words'].apply(lambda x: x.split())
df

,report,report words
year,,
2023,新华社北京3月14日电政府工作报告 ——2023年3月5日在第十四届全国人民代表大会第一...,"[新华社, 北京, 月, 日电, 政府, 工作, 报告, 年月日, 在, 第十四届, 全国人..."
2022,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协委员提出意见。 ...,"[各位, 代表, 现在, 我, 代表, 国务院, 向, 大会, 报告, 政府, 工作, 请予..."
2021,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协委员提出意见。 ...,"[各位, 代表, 现在, 我, 代表, 国务院, 向, 大会, 报告, 政府, 工作, 请予..."
2020,新华社北京5月29日电 政府工作报告 ——2020年5月22日在第十三届全国人民代表...,"[新华社, 北京, 月, 日电, 政府, 工作, 报告, 年月日, 在, 第十三届, 全国人..."
2019,新华社北京3月16日电 政府工作报告 ——2019年3月5日在第十三届全国人民代表...,"[新华社, 北京, 月, 日电, 政府, 工作, 报告, 年月日, 在, 第十三届, 全国人..."
2018,各位代表： 现在，我代表国务院，向大会报告过去五年政府工作，对今年工作提出建议，请予审...,"[各位, 代表, 现在, 我, 代表, 国务院, 向, 大会, 报告, 过去, 五年, 政府..."
2017,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协各位委员提出...,"[各位, 代表, 现在, 我, 代表, 国务院, 向, 大会, 报告, 政府, 工作, 请予..."
2016,政府工作报告 ——2016年3月5日在第十二届全国人民代表大会第四次会议上 国务院总理...,"[政府, 工作, 报告, 年月日, 在, 第十二届, 全国人民代表大会, 第四次, 会议, ..."
2015,国务院总理 李克强 各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并...,"[国务院, 总理, 李克强, 各位, 代表, 现在, 我, 代表, 国务院, 向, 大会, ..."


### LDA

In [42]:
dictionary = corpora.Dictionary(df['report words'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['report words']]     # 根据分词结果创建语料库

In [43]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [44]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.052*"的" + 0.039*"和" + 0.013*"发展" + 0.013*"要" + 0.009*"建设"')
(1, '0.026*"和" + 0.025*"的" + 0.016*"发展" + 0.008*"建设" + 0.008*"推进"')
(2, '0.065*"的" + 0.014*"在" + 0.014*"和" + 0.012*"年" + 0.011*"了"')
(3, '0.001*"的" + 0.001*"和" + 0.000*"发展" + 0.000*"要" + 0.000*"经济"')
(4, '0.093*"的" + 0.028*"和" + 0.014*"在" + 0.011*"了" + 0.010*"是"')


In [45]:
df['report words'].iloc[0]

['新华社',
 '北京',
 '月',
 '日电',
 '政府',
 '工作',
 '报告',
 '年月日',
 '在',
 '第十四届',
 '全国人民代表大会',
 '第一次',
 '会议',
 '上',
 '国务院',
 '总理',
 '李克强',
 '月',
 '日',
 '李克强',
 '代表',
 '国务院',
 '在',
 '十四届',
 '全国人大',
 '一次',
 '会议',
 '上作',
 '政府',
 '工作',
 '报告',
 '新华社',
 '记者',
 '饶',
 '爱民',
 '摄',
 '各位',
 '代表',
 '本届',
 '政府',
 '任期',
 '即将',
 '结束',
 '现在',
 '我',
 '代表',
 '国务院',
 '向',
 '大会',
 '报告',
 '工作',
 '请予',
 '审议',
 '并',
 '请',
 '全国政协',
 '委员',
 '提出',
 '意见',
 '一',
 '过去',
 '一年',
 '和',
 '五年',
 '工作',
 '回顾',
 '年',
 '是',
 '党和国家',
 '历史',
 '上',
 '极为重要',
 '的',
 '一年',
 '党',
 '的',
 '二十大',
 '胜利',
 '召开',
 '描绘',
 '了',
 '全面',
 '建设',
 '社会主义',
 '现代化',
 '国家',
 '的',
 '宏伟蓝图',
 '面对',
 '风高浪',
 '急',
 '的',
 '国际',
 '环境',
 '和',
 '艰巨',
 '繁重',
 '的',
 '国内',
 '改革',
 '发展',
 '稳定',
 '任务',
 '以',
 '习近平',
 '同志',
 '为',
 '核心',
 '的',
 '党中央',
 '团结',
 '带领',
 '全国',
 '各族人民',
 '迎难而上',
 '全面落实',
 '疫情',
 '要',
 '防住',
 '经济',
 '要',
 '稳住',
 '发展',
 '要',
 '安全',
 '的',
 '要求',
 '加大',
 '宏观调控',
 '力度',
 '实现',
 '了',
 '经济',
 '平稳',
 '运行',
 '发展',
 '质量',
 '稳步',
 '提升',
 '社会',
 '大局',
 '保持

In [46]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9998971819877625	 
Topic: 0.026*"和" + 0.025*"的" + 0.016*"发展" + 0.008*"建设" + 0.008*"推进" + 0.007*"要" + 0.007*"加强" + 0.007*"经济" + 0.007*"改革" + 0.007*"等"


In [48]:
documents = df['report words'].values

In [49]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(1, 0.9998972)]

Document 2:
[(1, 0.9998892)]

Document 3:
[(1, 0.9998913)]

Document 4:
[(1, 0.9998349)]

Document 5:
[(1, 0.99990815)]

Document 6:
[(1, 0.99990803)]

Document 7:
[(1, 0.99990237)]

Document 8:
[(1, 0.9999065)]

Document 9:
[(1, 0.99984556)]

Document 10:
[(1, 0.9983449)]



### 可视化

In [50]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) 
pyLDAvis.display(lda_vis)

In [51]:
# 导出可视化结果到html
# pyLDAvis.save_html(lda_vis, 'lda_visualization.html')